In [10]:
import pandas as pd
import numpy as np
import math

In [11]:
articles_df = pd.read_csv('data/shared_articles.zip')
articles_df = articles_df[articles_df['eventType'] == 'CONTENT SHARED']

In [12]:
interactions_df = pd.read_csv('data/users_interactions.zip')

In [13]:
interactions_df.personId = interactions_df.personId.astype(str)
interactions_df.contentId = interactions_df.contentId.astype(str)
articles_df.contentId = articles_df.contentId.astype(str)

In [14]:
event_type_strength = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}

In [15]:
interactions_df['eventStrength'] = interactions_df.eventType.apply(lambda x: event_type_strength[x])

In [16]:
users_interactions_count_df = (
    interactions_df
    .groupby(['personId', 'contentId'])
    .first()
    .reset_index()
    .groupby('personId').size())

users_with_enough_interactions_df = \
    users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[['personId']]

In [17]:
interactions_from_selected_users_df = interactions_df.loc[np.in1d(interactions_df.personId,
            users_with_enough_interactions_df)]

In [18]:
def smooth_user_preference(x):
    return math.log(1+x, 2)
    
interactions_full_df = (
    interactions_from_selected_users_df
    .groupby(['personId', 'contentId']).eventStrength.sum()
    .apply(smooth_user_preference)
    .reset_index().set_index(['personId', 'contentId'])
)
interactions_full_df['last_timestamp'] = (
    interactions_from_selected_users_df
    .groupby(['personId', 'contentId'])['timestamp'].max()
)
        
interactions_full_df = interactions_full_df.reset_index()

In [19]:
from sklearn.model_selection import train_test_split

split_ts = 1475519545
interactions_train_df = interactions_full_df.loc[interactions_full_df.last_timestamp < split_ts].copy()
interactions_test_df = interactions_full_df.loc[interactions_full_df.last_timestamp >= split_ts].copy()


 6. Практика

Задание 6.1
1 point possible (graded)
Найдите оценку взаимодействия пользователя с ID -1032019229384696495 со статьёй с ID 943818026930898372. Результат округлите до двух знаков после точки-разделителя.

Примечание. Здесь и далее (пока не будет указано иное) необходимо работать с обучающей выборкой.

In [26]:
pivot_df = pd.pivot_table(interactions_train_df,  values='eventStrength', index=['personId'],
                       columns=['contentId'], aggfunc="sum", fill_value=0)
pivot_df['943818026930898372'].loc['-1032019229384696495']

2.321928094887362

Задание 6.2
1 point possible (graded)
Найдите среднее арифметическое всех чисел в получившемся массиве. Результат округлите до трёх знаков после точки-разделителя.

In [28]:
pivot_df.mean().mean()

0.016668620737604056

In [39]:
ratings_m = pivot_df.values
ratings_m

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

Задание 6.3
1 point possible (graded)
Постройте матрицу схожести. Для этого вычислите все попарные коэффициенты корреляции для матрицы, полученной в предыдущем задании. Для каждой пары учитывайте только ненулевые значения (так как нулевые обозначают отсутствие взаимодействия и не интересуют нас). Выведите результат, полученный в ячейке с третьим индексом по строкам и сороковым — по столбцам. Ответ округлите до двух знаков после точки-разделителя.

In [40]:
similarity_users = np.zeros((len(ratings_m), len(ratings_m)))
for i in (range(len(ratings_m)-1)):
    for j in range(i+1, len(ratings_m)):
     
        mask_uv = (ratings_m[i] != 0) & (ratings_m[j] != 0)
        ratings_v = ratings_m[i, mask_uv]
        ratings_u = ratings_m[j, mask_uv]

        similarity_users[i,j] = np.corrcoef(ratings_v, ratings_u)[0, 1]
        similarity_users[j,i] = similarity_users[i,j]

similarity_users[3,40]

c:\Projects\DS\lessons\.venv\Lib\site-packages\numpy\lib\function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
c:\Projects\DS\lessons\.venv\Lib\site-packages\numpy\core\_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
c:\Projects\DS\lessons\.venv\Lib\site-packages\numpy\lib\function_base.py:2889: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
c:\Projects\DS\lessons\.venv\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
c:\Projects\DS\lessons\.venv\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
c:\Projects\DS\lessons\.venv\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Projects\DS\lessons\.venv\Lib\site-packages

-0.3333333333333333

In [41]:
similarity_users

array([[ 0.        ,         nan,         nan, ...,         nan,
                nan,         nan],
       [        nan,  0.        ,  0.41742881, ..., -0.82922103,
                nan,         nan],
       [        nan,  0.41742881,  0.        , ..., -1.        ,
                nan,         nan],
       ...,
       [        nan, -0.82922103, -1.        , ...,  0.        ,
                nan,         nan],
       [        nan,         nan,         nan, ...,         nan,
         0.        ,         nan],
       [        nan,         nan,         nan, ...,         nan,
                nan,  0.        ]])

Для каждого пользователя:

Найти пользователей с похожестью больше 0.
Для каждой статьи вычислить долю пользователей (среди выделенных на первом шаге), которые взаимодействовали со статьёй.
Порекомендовать статьи (не более 10) с наибольшими долями со второго шага (среди тех, которые пользователь ещё не видел).

Задание 6.4
1 point possible (graded)
Постройте рекомендательную систему по алгоритму, описанному выше. Найдите первую рекомендацию для строки 35 (если считать с нуля).

In [43]:
interactions = (
    interactions_train_df
    .groupby('personId')['contentId'].agg(lambda x: list(x))
    .reset_index()
    .rename(columns={'contentId': 'true_train'})
    .set_index('personId')
)
 
interactions['true_test'] = (
    interactions_test_df
    .groupby('personId')['contentId'].agg(lambda x: list(x))
)
 
interactions['true_test'] = [ [] if x is np.NaN else x for x in interactions['true_test'] ]

prediction_user_based = []
for i in range(len(similarity_users)):
    users_sim = similarity_users[i] > 0
    if not any(users_sim):
        prediction_user_based.append([])
    else:
        tmp_recommend = np.argsort(ratings_m[users_sim].sum(axis=0))[::-1]
        tmp_recommend = pivot_df.columns[tmp_recommend]
        recommend = np.array(tmp_recommend)[~np.in1d(tmp_recommend, interactions.iloc[i]["true_train"])][:10]
        prediction_user_based.append(list(recommend))
interactions['prediction_user_based'] = prediction_user_based
prediction_user_based[35][0]

'-5148591903395022444'

In [44]:
interactions

,true_train,true_test,prediction_user_based
personId,,,
-1007001694607905623,"[-5065077552540450930, -793729620925729327]","[-6623581327558800021, 1469580151036142903, 72...",[]
-1032019229384696495,"[-1006791494035379303, -1039912738963181810, -...","[-1415040208471067980, -2555801390963402198, -...","[-8208801367848627943, 8657408509986329668, -1..."
-108842214936804958,"[-1196068832249300490, -133139342397538859, -1...","[-2780168264183400543, -3060116862184714437, -...","[8657408509986329668, -8208801367848627943, -8..."
-1130272294246983140,"[-1150591229250318592, -1196068832249300490, -...","[-1606980109000976010, -1663441888197894674, -...","[2857117417189640073, 8657408509986329668, -82..."
-1160159014793528221,"[-133139342397538859, -387651900461462767, 377...",[-3462051751080362224],"[8657408509986329668, 7544768317373280661, -68..."
...,...,...,...
953707509720613429,"[-1068603220639552685, -2358756719610361882, -...","[-2402288292108892893, -5813211845057621660, -...","[-133139342397538859, -5488842573681626972, 82..."
983095443598229476,"[-133139342397538859, -8742648016180281673]","[-14569272361926584, -1572252285162838958, -18...","[-1297580205670251233, -4571929941432664145, -..."
989049974880576288,"[-133139342397538859, -2038869595290705317, -2...","[-6289909056857931861, -7047448754687279385, -...","[-5148591903395022444, 8657408509986329668, -6..."


In [45]:
def calc_precision(column):
    return ( interactions.apply(  lambda row:len(set(row['true_test']).intersection(
                set(row[column]))) /min(len(row['true_test']) + 0.001, 10.0), axis=1)).mean()

Задание 6.5
1 point possible (graded)
Вычислите точность полученного предсказания. Ответ округлите до трёх знаков после точки-разделителя.

In [46]:
round(calc_precision('prediction_user_based'), 3)

0.005

Задание 6.6
1 point possible (graded)
Теперь реализуем рекомендательную систему с использованием SVD.

Разложите матрицу взаимодействий пользователей со статьями с помощью функции svd из модуля scipy. Найдите максимальное значение в получившейся матрице U. Результат округлите до двух знаков после точки-разделителя.

In [48]:
from scipy.linalg import svd

In [51]:
U, sigma, V = svd(pivot_df)
U.max()


0.7071067811865468

In [52]:
k = 100
s = np.diag(sigma[:k])
U = U[:, 0:k]
V = V[0:k, :]

round(s.sum(), 2)

2096.43

Задание 6.8
1 point possible (graded)
Вычислите качество полученного предсказания, используя всё ту же метрику точности. Ответ округлите до трёх знаков после точки-разделителя.

In [54]:
new_ratings = pd.DataFrame(
    U.dot(s).dot(V), index=pivot_df.index, columns=pivot_df.columns
)
top_k = 10
predictions = []

for personId in interactions.index:
    prediction = (
        new_ratings.loc[personId].sort_values(ascending=False).index.values
    )

    predictions.append(
        list(
            prediction[
                ~np.in1d(prediction, interactions.loc[personId, "true_train"])
            ]
        )[:top_k]
    )

interactions["prediction_svd"] = predictions

calc_precision("prediction_svd")

0.012023152289763838

Возьмите матрицу, подготовленную в задании 6.1. Преобразуйте её в разреженную матрицу:

from scipy.sparse import csr_matrix
ratings_matrix = csr_matrix(ratings)
Воспользовавшись функцией random_train_test_split() из библиотеки lightfm, разделите данные на валидационную и обучающую выборки в соотношении 1:2 (30% на валидационную выборку, 70% на обучающую). В качестве значения параметра random_state возьмите число 13.

Обучите модель LightFM со 100 компонентами, параметром random_state = 13, темпом обучения 0.05 и функцией потерь 'warp'. Обратите внимание на то, что так как в данном случае у нас нет item-признаков, то параметр item_features задавать не нужно.

Вычислите показатель точности (precision@k) при k = 10. Ответ округлите до двух знаков после точки-разделителя.

In [ ]:
from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k, recall_at_k  
from scipy.sparse import csr_matrix

ratings_matrix = csr_matrix(ratings) # передаём в качестве аргументов в функцию выставленный рейтинг (это будут значения матрицы), а также id пользователя и id книги (это будут индексы для строк и столбцов матрицы)
model = LightFM(loss='warp', #определяем функцию потерь
                random_state=13, #фиксируем случайное разбиение
                learning_rate=0.05, #темп обучения
                no_components=100) #размерность вектора для представления данных в модели

train,test = random_train_test_split(ratings_matrix, test_percentage=0.3, random_state=13)
model.fit(train)
prec_score = precision_at_k(model, test).mean()


print(round(prec_score, 2))